In [1]:
import sys
sys.path.insert(1, "../../")
sys.path.insert(1, "../../Models/")
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from yolov2 import YOLOv2D19 as YOLOv2
from detection_datasets import VOCDatasetV2
from torch import optim
from loss import YOLOv2Loss
from train import *
import torch.optim.lr_scheduler as lr_scheduler
from data_preprocessing import get_norms
import pickle
with open('../../Models/anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

C:\Vstanovlene\Anaconda Distribution\envs\new_gpu_env\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
device = torch.device('cuda:0')
dtype=torch.float32

In [3]:
norms = get_norms('../../../datasets/VOCdevkit/trainval_norms.json')
means = norms['means']
stds = norms['stds']

In [4]:
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
import gc

# Invoke garbage collector
gc.collect()

# Clear GPU cache
torch.cuda.empty_cache()
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

Allocated memory: 0.00 MB
Allocated memory: 0.00 MB


In [5]:
transforms = A.Compose([
    A.Resize(width=416, height=416),
    # A.VerticalFlip(p=1.0),
    A.Normalize(mean=means, std=stds),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../../datasets/VOCdevkit/', 
                         subsets = [('VOC2012', 'trainval')],
                         scales=[13], anchors=anchors, transforms=transforms, 
                         dtype=dtype, device=device)
val_set = VOCDatasetV2(devkit_path = '../../../datasets/VOCdevkit/', 
                       subsets = [('VOC2007', 'val')],
                       scales=[13], anchors=anchors, transforms=transforms, 
                       dtype=dtype, device=device)

True ../../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt
True ../../../datasets/VOCdevkit/VOC2007\ImageSets\Main\val.txt


In [6]:
train_set = Subset(train_set, list(range(0, 100)))
val_set = Subset(val_set, list(range(1, 10)))

In [7]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

In [28]:
loss_fn = YoloLoss(anchors=torch.tensor(anchors))

In [29]:
model = YOLOv2(state_dict_path='../../Models/darknet19_72.96.pth', device=device, dtype=dtype, num_anchors=3)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Detection experiments\YOLOv2\../../Models\yolov2.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state

In [30]:
epochs=100
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [31]:
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=1.0)

In [32]:
scaler = torch.amp.GradScaler("cuda")

In [40]:
anchors*3

[[0.32882816561531697, 0.6554346360225188],
 [0.07522471620262354, 0.1241032298491792],
 [0.7862514755140577, 0.8175396064741282],
 [0.32882816561531697, 0.6554346360225188],
 [0.07522471620262354, 0.1241032298491792],
 [0.7862514755140577, 0.8175396064741282],
 [0.32882816561531697, 0.6554346360225188],
 [0.07522471620262354, 0.1241032298491792],
 [0.7862514755140577, 0.8175396064741282]]

In [39]:
torch.tensor(anchors).reshape(1, 3, 1, 1, 2)

tensor([[[[[0.3288, 0.6554]]],


         [[[0.0752, 0.1241]]],


         [[[0.7863, 0.8175]]]]])

In [33]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, scaler, 
                                save_grad=False, outputs_path='../log/YOLOv2/training/')

2025-01-23 21:42:15.563512 Epoch 1 
scaler used


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, scaler, 
                                save_grad=False, outputs_path='../log/YOLOv2/training/')

Results when loss_fn is MSE, 13x13x125, full sets

In [25]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, scaler, 
                                save_grad=False, outputs_path='../log/YOLOv2/training/')

2024-12-23 08:42:34.189888 Epoch 1 
2024-12-23 08:42:50.410643 Batch 30 
2024-12-23 08:43:07.046975 Batch 60 
2024-12-23 08:43:23.138869 Batch 90 
2024-12-23 08:43:39.060661 Batch 120 
2024-12-23 08:43:55.065185 Batch 150 
2024-12-23 08:44:11.294661 Batch 180 
2024-12-23 08:44:27.271376 Batch 210 
2024-12-23 08:44:43.124076 Batch 240 
2024-12-23 08:44:59.231072 Batch 270 
2024-12-23 08:45:15.154746 Batch 300 
2024-12-23 08:45:31.198184 Batch 330 
2024-12-23 08:45:47.260832 Batch 360 
[Train] Loss per batch: 0.0364
2024-12-23 08:45:58.658664 Batch 30 
2024-12-23 08:46:09.537877 Batch 60 
[Val] loss per batch: 0.0248
Epoch 1: SGD lr 0.0100 -> 0.0100
2024-12-23 08:46:16.661041 Epoch 2 
2024-12-23 08:46:32.511914 Batch 30 
2024-12-23 08:46:48.611562 Batch 60 
2024-12-23 08:47:04.673063 Batch 90 
2024-12-23 08:47:20.756701 Batch 120 
2024-12-23 08:47:37.109335 Batch 150 
2024-12-23 08:47:53.084386 Batch 180 
2024-12-23 08:48:08.983728 Batch 210 
2024-12-23 08:48:24.952682 Batch 240 
2024-12

KeyboardInterrupt: 